# Data Cleaning and Preprocessing

## Overview
This notebook covers essential steps for preparing raw data before analysis. The goal is to ensure that datasets are structured, free from inconsistencies, and ready for merging or visualization. The steps include:

- Handling missing values
- Formatting column names
- Filtering and removing duplicates
- Creating new features if necessary

Data cleaning is the first step in any data analysis pipeline, ensuring the integrity and reliability of insights drawn from the dataset.

---

## Importing Necessary Libraries
The following libraries are used for data manipulation and visualization:
- `pandas`: For working with structured data (tables, CSV, Excel)
- `numpy`: For numerical computations
- `matplotlib` & `seaborn`: For potential visual exploration

In [ ]:
# Import Libraries
import os
import pandas as pd
import numpy as np
import chardet

# Visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

from IPython.display import display, HTML

# Force Jupyter Notebook to use all available horizontal space
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.width', 1000)          # Set width to a large number
pd.set_option('display.max_colwidth', None)     # Show full column content if needed
pd.set_option('display.float_format', lambda x: f"{x:,.2f}".replace(',', ' '))  # Format numbers with 2 decimal places

In [ ]:
# Get the relative path of the directory where this script/notebook is located.
script_dir = os.getcwd()  # or wherever your notebook is running

# Go one level up (to the parent folder) and then into "02 Data".
data_folder = os.path.join(script_dir, '..', '02 Data')
input_path = os.path.join(data_folder, 'Original_data')
output_path = os.path.join(data_folder, 'Processed_data')
summary_report_path = os.path.join(output_path, 'summary_report.txt')

# Create the output folder if it doesn't exist
os.makedirs(output_path, exist_ok=True)

print("Data folder:", data_folder)
print("Input path:", input_path)
print("Output path:", output_path)
print("Summary report path:", summary_report_path)

In [ ]:
# Verify the input folder exists and list available files.
if not os.path.exists(input_path):
    print(f"Error: The folder '{input_path}' does not exist. Please ensure the base folder is correct.")
else:
    available_files = [f for f in os.listdir(input_path)]
    print("Available files in the input folder:")
    for idx, f in enumerate(available_files, start=1):
        print(f"{idx}. {f}")
    
    file_numbers_input = input(
        "\nEnter the file numbers to process (comma-separated), or leave blank to process all files: "
    ).strip()
    
    if file_numbers_input:
        try:
            indices = [int(num.strip()) for num in file_numbers_input.split(',') if num.strip()]
            # Validate indices and build the list of selected files.
            files_list = [available_files[i-1] for i in indices if 1 <= i <= len(available_files)]
            if not files_list:
                print("No valid file numbers were entered.")
        except ValueError:
            print("Error: Please enter valid numbers separated by commas.")
            files_list = []
    else:
        files_list = available_files

    print("\nFiles selected for processing:", files_list)

In [ ]:
# Starting report generation
summary_lines = []
current_file = files_list[0]
file_path = os.path.join(input_path, current_file)

In [ ]:
def preview_csv_with_delimiters(file_path, 
                                fallback_encodings=['latin1', 'ISO-8859-1', 'cp1252'], 
                                possible_delimiters=[',', ';', '\t', '|']):
    """
    Detects the file encoding, then previews the first 3 rows of the CSV using various delimiters.
    Returns the detected encoding and a dictionary of previews keyed by delimiter.
    """
    # Detect encoding using a sample from the file.
    with open(file_path, 'rb') as f:
        rawdata = f.read(100000)  # Read first 100k bytes
    detection = chardet.detect(rawdata)
    encoding = detection.get('encoding', 'utf-8')
    print(f"Detected encoding: {encoding}\n")
    
    previews = {}
    for delim in possible_delimiters:
        print(f"Preview using delimiter {repr(delim)}:")
        try:
            # Read only the first 3 rows for preview
            df_preview = pd.read_csv(file_path, encoding=encoding, sep=delim, nrows=3)
            previews[delim] = df_preview
            print(df_preview)
        except Exception as e:
            print(f"Failed with delimiter {repr(delim)}. Error: {e}")
        print("-" * 50 + "\n")
    return encoding, previews

# Dictionary to store the DataFrames for each file.
df = {}

for file in files_list:
    file_path = os.path.join(input_path, file)
    if os.path.exists(file_path):
        if file.endswith('.csv'):
            print(f"Processing CSV file: {file}")
            # Preview the CSV with different delimiters.
            encoding, previews = preview_csv_with_delimiters(file_path)
            
            # List the delimiter options with numbers.
            possible_delimiters = [',', ';', '\t', '|']
            print("Select the correct delimiter from the options below:")
            for idx, delim in enumerate(possible_delimiters, start=1):
                print(f"{idx}. {repr(delim)}")
            
            # Ask the user to choose the correct delimiter by number.
            while True:
                try:
                    selected_number = int(input("Enter the number corresponding to the correct delimiter: "))
                    if 1 <= selected_number <= len(possible_delimiters):
                        selected_delim = possible_delimiters[selected_number - 1]
                        break
                    else:
                        print("Invalid number. Please choose a valid option.")
                except ValueError:
                    print("Please enter a valid integer.")
            
            # Now load the full CSV using the selected delimiter.
            try:
                df_csv = pd.read_csv(file_path, encoding=encoding, sep=selected_delim)
                df[file] = df_csv
                print(f"Successfully loaded '{file}' with delimiter {repr(selected_delim)} " \
                      f"(rows: {df_csv.shape[0]}, columns: {df_csv.shape[1]})")
            except Exception as e:
                print(f"Error loading CSV file {file} with delimiter {repr(selected_delim)}: {e}")
                continue
        elif file.endswith('.pkl'):
            try:
                df[file] = pd.read_pickle(file_path)
                print(f"Loaded pickle file '{file}' (rows: {df[file].shape[0]}, columns: {df[file].shape[1]})")
            except Exception as e:
                print(f"Error reading pickle file {file}: {e}")
                continue
        else:
            print(f"Skipping unsupported file format: {file}")
            continue
        
        print("=" * 100 + "\n")
    else:
        print(f"File {file} not found and will be skipped.")

# Logging details.
report_details = [f"File: {current_file}"]
report_details.append(f"Total loaded files: {len(df)}")
modifications = []

In [ ]:
# present all imported DataFrames

for file_name, data in df.items():
    html = data.to_html(max_rows=2, max_cols=30)
    display(HTML(f'<h4>{file_name}</h4><div style="overflow-x: auto; width:100%;">{html}</div>'))

In [ ]:
df[current_file].head()

In [ ]:
df[current_file].shape

## Handling Missing Values
Missing data can affect analysis accuracy. This section explores strategies such as:
- Removing rows/columns with excessive missing values
- Imputing missing values based on statistical methods
- Using placeholders for unknown values

In [ ]:
# Data Customization – Exclude Columns
df[current_file].head()
data = df[current_file]
data.head()
exclude_cols_input = input("\nEnter columns to exclude (comma-separated), or press Enter to skip: ").strip()
if exclude_cols_input:
    exclude_cols = [col.strip() for col in exclude_cols_input.split(',') if col.strip()]
    data = data.drop(columns=exclude_cols, errors='ignore')
    modifications.append(f"Excluded columns: {', '.join(exclude_cols)}")
    print(f"Columns excluded: {exclude_cols}")
else:
    print("No columns were excluded.")

# Update the DataFrame in the dictionary.
df[current_file] = data

## Formatting and Standardizing Column Names
To ensure consistency across datasets, column names are standardized:
- Converted to lowercase
- Replacing spaces with underscores
- Removing special characters

In [ ]:
# Data Customization – Rename Columns
while True:
    print("\nCurrent column names:")
    print(list(data.columns))
    col_to_rename = input("Enter column name to rename, or press Enter to stop renaming: ").strip()
    if not col_to_rename:
        break
    if col_to_rename in data.columns:
        new_name = input(f"Enter new name for column '{col_to_rename}': ").strip()
        data.rename(columns={col_to_rename: new_name}, inplace=True)
        modifications.append(f"Renamed column '{col_to_rename}' to '{new_name}'")
        print(f"Renamed '{col_to_rename}' to '{new_name}'")
    else:
        print(f"Column '{col_to_rename}' not found.")
        
df[current_file] = data

In [ ]:
# Example function to clean currency-like strings and convert to float
def parse_currency_string(value):
    """
    Attempts to remove currency symbols, spaces, and convert commas to dots,
    then parses the result as a float. Leaves NaNs as NaN.
    """
    if pd.isna(value):
        return value  # Keep NaNs as is
    # Remove '$' if present (or other currency symbols)
    value = str(value).replace('$', '')
    # Remove spaces
    value = value.replace(' ', '')
    # Replace comma with dot (assuming comma decimal format)
    value = value.replace(',', '.')
    # Convert to float
    return float(value)

# List of available data types (key + explanation)
available_dtypes = [
    ('int', "Integer (no decimal part). E.g. 42"),
    ('float', "Floating-point number (decimal allowed). E.g. 3.14"),
    ('str', "String (text). E.g. 'Hello world'"),
    ('bool', "Boolean (True or False)"),
    ('datetime64[ns]', "Date and time in Pandas datetime format"),
    ('category', "Categorical data (saves memory if repetitive values)"),
]

# Data Customization – Change Data Types by column number
while True:
    print("\nCurrent data types:")
    print(data.dtypes)
    
    # Number each column
    columns_list = list(data.columns)
    print("\nColumns:")
    for i, col in enumerate(columns_list, start=1):
        print(f"{i}. {col} (current dtype: {data[col].dtype})")
    
    # Ask user which column to convert by number
    col_number_input = input("\nEnter the column number to change data type, or press Enter to stop:\n").strip()
    if not col_number_input:
        break  # Stop if user presses Enter without a choice
    
    try:
        col_number = int(col_number_input)
        if 1 <= col_number <= len(columns_list):
            col_to_cast = columns_list[col_number - 1]
        else:
            print("Invalid column number. Please try again.")
            continue
    except ValueError:
        print("Please enter a valid integer for the column number.")
        continue
    
    # Show available data types by number
    print("\nAvailable data types:")
    for idx, (dt_key, dt_expl) in enumerate(available_dtypes, start=1):
        print(f"{idx}. {dt_key} - {dt_expl}")
    
    # Ask user for the new data type by number
    dtype_choice = input(f"\nEnter the number corresponding to the desired data type for '{col_to_cast}':\n").strip()
    try:
        dtype_choice_num = int(dtype_choice)
        if 1 <= dtype_choice_num <= len(available_dtypes):
            new_dtype = available_dtypes[dtype_choice_num - 1][0]
        else:
            print("Invalid number. Please choose a valid option.")
            continue
    except ValueError:
        print("Please enter a valid integer.")
        continue
    
    # Try direct casting first
    try:
        data[col_to_cast] = data[col_to_cast].astype(new_dtype)
        modifications.append(f"Changed data type of '{col_to_cast}' to {new_dtype}")
        print(f"Changed data type of '{col_to_cast}' to {new_dtype}")
    except Exception as e:
        print(f"Failed to change data type of '{col_to_cast}' to {new_dtype}: {e}")
        
        # Attempt auto-clean only if user wants int or float
        if new_dtype in ["int", "float"]:
            print(f"\nAttempting to auto-clean the '{col_to_cast}' column for {new_dtype} conversion...")

            try:
                # 1. Clean the column (remove currency symbols, fix decimal separators, etc.)
                temp_col = data[col_to_cast].apply(parse_currency_string)
                
                # 2. Handle missing values before casting to int/float
                missing_count = temp_col.isna().sum()
                if missing_count > 0:
                    print(f"Found {missing_count} missing (NaN) values in '{col_to_cast}'.")
                    print("How would you like to handle these missing values?")
                    print("1. Drop rows with missing values")
                    print("2. Fill missing values with 0")
                    print("3. Fill missing values with a custom value")
                    print("4. Leave them as NaN (only works for float)")
                    mv_choice = input("Enter the number of your choice (1/2/3/4): ").strip()
                    
                    if mv_choice == "1":
                        temp_col = temp_col.dropna()
                        print("Dropped rows with missing values.")
                    elif mv_choice == "2":
                        temp_col = temp_col.fillna(0)
                        print("Filled missing values with 0.")
                    elif mv_choice == "3":
                        fill_val = input("Enter the value to fill missing values: ")
                        # Convert fill_val to the appropriate numeric type
                        if new_dtype == "float":
                            fill_val = float(fill_val)
                        elif new_dtype == "int":
                            fill_val = int(float(fill_val))  # in case user typed "3.0"
                        temp_col = temp_col.fillna(fill_val)
                        print(f"Filled missing values with '{fill_val}'.")
                    elif mv_choice == "4":
                        if new_dtype == "float":
                            print("Leaving missing values as NaN.")
                        else:
                            print("Cannot leave NaN if converting to int. Attempting fill with 0.")
                            temp_col = temp_col.fillna(0)
                    else:
                        print("Invalid choice. Leaving missing values as NaN for now.")
                
                # 3. Convert to float or int as requested
                if new_dtype == "int":
                    temp_col = temp_col.astype(int)
                else:
                    temp_col = temp_col.astype(float)
                
                # 4. Show side-by-side comparison for first 5 rows
                comparison_df = pd.DataFrame({
                    "original": data[col_to_cast].head(5),
                    "converted": temp_col.head(5)
                })
                
                print("\nPreview of original vs. converted values (first 5 rows):")
                print(comparison_df)
                
                # 5. Prompt user confirmation
                confirm = input("\nDoes this look correct? (y/n): ").strip().lower()
                if confirm == "y":
                    # If we dropped rows, we need to align the main DataFrame with temp_col’s index
                    data = data.reindex(temp_col.index)  # In case some rows were dropped
                    data[col_to_cast] = temp_col
                    modifications.append(f"Auto-cleaned and changed data type of '{col_to_cast}' to {new_dtype}")
                    print(f"Successfully auto-cleaned and changed data type of '{col_to_cast}' to {new_dtype}")
                else:
                    print("No changes applied.")
            
            except Exception as e2:
                print(f"Auto-cleaning also failed: {e2}")
        else:
            print("Auto-cleaning is only implemented for int or float conversions.")

# Finally, store the updated DataFrame back if needed
df[current_file] = data

In [ ]:
# # Checking for missing values in the dataset
print("\nChecking for missing values...")
missing_summary = data.isnull().sum()
missing_summary = missing_summary[missing_summary > 0]
rows_before = len(data)

if not missing_summary.empty:
    print("Columns with missing values and their counts:")
    print(missing_summary)
    
    print("\nColumn statistics:")
    display(data.describe())
    
    # Loop through each column that has missing values
    for column in missing_summary.index:
        # Display the first 5 rows where the current column has missing values
        print(f"\nFirst 5 rows where '{column}' is missing:")
        display(data[data[column].isnull()].head())
        
        fill_method = input(
            f"Enter method to handle missing values for '{column}' "
            "(mean, median, drop, or custom value), or press Enter to skip: "
        ).strip()
        
        if fill_method == 'mean':
            data[column] = data[column].fillna(data[column].mean())
            modifications.append(f"Filled missing values in '{column}' with mean")
        
        elif fill_method == 'median':
            data[column] = data[column].fillna(data[column].median())
            modifications.append(f"Filled missing values in '{column}' with median")
        
        elif fill_method == 'drop':
            data.dropna(subset=[column], inplace=True)
            modifications.append(f"Dropped rows with missing values in '{column}'")
        
        elif fill_method:
            try:
                if data[column].dtype.kind in 'fc':  # numeric types
                    value = float(fill_method)
                else:
                    value = fill_method
                
                data[column] = data[column].fillna(value)
                modifications.append(f"Filled missing values in '{column}' with custom value: {value}")
            
            except ValueError:
                print(f"Invalid custom value for '{column}', skipping...")
else:
    print("No missing values detected.")

rows_after = len(data)
report_details.append(f"Rows dropped due to missing values: {rows_before - rows_after}")

df[current_file] = data

## Removing Duplicates
Duplicate records can distort analysis results. This step identifies and removes any redundant entries to maintain dataset integrity.

In [ ]:
# Duplicate Row Management

print("\nChecking for duplicate rows...")
duplicates = data.duplicated().sum()
print(f"Found {duplicates} duplicate rows.")
report_details.append(f"Number of duplicate rows: {duplicates}")

if duplicates > 0:
    print("Preview of duplicate rows:")
    display(data[data.duplicated()].head())
    drop_dup = input("Do you want to drop duplicates? (yes/no): ").strip().lower()
    if drop_dup == 'yes':
        rows_before_dup = len(data)
        data.drop_duplicates(inplace=True)
        rows_after_dup = len(data)
        modifications.append("Dropped duplicate rows")
        report_details.append(f"Rows dropped due to duplicates: {rows_before_dup - rows_after_dup}")
        print("Duplicates dropped.")
    else:
        print("Duplicates not dropped.")
        
df[current_file] = data

In [ ]:
# Outlier Detection

print("\nDetecting outliers in numeric columns...")
outliers_info = {}
for col in data.select_dtypes(include=['number']).columns:
    q1 = data[col].quantile(0.25)
    q3 = data[col].quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    outlier_rows = data[(data[col] < lower_bound) | (data[col] > upper_bound)]
    if not outlier_rows.empty:
        outliers_info[col] = len(outlier_rows)
        print(f"Column '{col}': {len(outlier_rows)} outlier rows detected.")
        display(outlier_rows.head())
        
if outliers_info:
    modifications.append(f"Outliers detected: {outliers_info}")
    report_details.append("Outlier detection completed.")
else:
    report_details.append("No outliers detected.")


In [ ]:
print(data.columns)

In [ ]:
df[current_file].shape

In [ ]:
# Ask user for file format preference: CSV or pkl
file_format = input("Enter desired output file format (csv or pkl): ").strip().lower()
while file_format not in ['csv', 'pkl']:
    file_format = input("Invalid format. Please enter 'csv' or 'pkl': ").strip().lower()

# Prompt the user for the file name (without extension)
output_filename = input("Enter the desired file name (without extension): ").strip()
output_file = os.path.join(output_path, f"{output_filename}.{file_format}")

# Save the processed DataFrame in the selected format
if file_format == 'csv':
    data.to_csv(output_file, index=False)
elif file_format == 'pkl':
    data.to_pickle(output_file)

print(f"\n✅ Processed file saved to: {output_file}")
report_details.append(f"Processed file saved to: {output_file}")
report_details.append(f"Total rows in the exported file: {len(data)}")

# Update the stored data frame for the current file
df[current_file] = data